In [1]:
from neuronx_distributed_inference.utils.testing import build_module

/opt/aws_neuronx_venv_pytorch_2_8_nxd_inference/lib/python3.10/site-packages/neuronx_distributed/parallel_layers/layers.py:14: DeprecationWarning: torch_neuronx.nki_jit is deprecated, use nki.jit instead.
  from .mappings import (
/opt/aws_neuronx_venv_pytorch_2_8_nxd_inference/lib/python3.10/site-packages/neuronx_distributed/parallel_layers/layers.py:14: DeprecationWarning: torch_neuronx.nki_jit is deprecated, use nki.jit instead.
  from .mappings import (
/opt/aws_neuronx_venv_pytorch_2_8_nxd_inference/lib/python3.10/site-packages/neuronx_distributed/parallel_layers/layers.py:14: DeprecationWarning: torch_neuronx.nki_jit is deprecated, use nki.jit instead.
  from .mappings import (
/opt/aws_neuronx_venv_pytorch_2_8_nxd_inference/lib/python3.10/site-packages/neuronx_distributed/modules/moe/blockwise.py:68: DeprecationWarning: torch_neuronx.nki_jit is deprecated, use nki.jit instead.
  component, error = import_nki(config)
/opt/aws_neuronx_venv_pytorch_2_8_nxd_inference/lib/python3.10/

In [2]:
import torch


def piecewise_safe_division(a, b):
    """
    Piecewise safe division of tensors a/b with special rules for division by zero.
    Unravels/flattens `a` and `b` if not already 1-D (handled by summary_stats).
    """
    is_zero = b == 0

    # Compute x/y where y != 0
    div = a / b

    # For x==0 & y==0 --> 0
    case1 = (a == 0) & is_zero

    # For x<0 & y==0 --> -inf
    case2 = (a < 0) & is_zero

    # For x>0 & y==0 --> inf
    case3 = (a > 0) & is_zero

    # Start with normal division
    c = div

    # Set 0 where x==0 & y==0
    c = torch.where(case1, torch.zeros_like(c), c)

    # Set -inf where x<0 & y==0
    c = torch.where(case2, torch.full_like(c, -float('inf')), c)

    # Set inf where x>0 & y==0
    c = torch.where(case3, torch.full_like(c, float('inf')), c)

    return c


def summary_stats(A, B):
    if isinstance(A, torch.Tensor) and isinstance(B, torch.Tensor):
        with torch.no_grad():
            # 1. Check matching shape
            if A.shape != B.shape:
                raise ValueError(f"Tensor shape mismatch: {A.shape} vs {B.shape}")

            # 2. Flatten inputs
            A = A.contiguous().view(-1)
            B = B.contiguous().view(-1)

            diff = A - B

            abs_error = torch.abs(diff)
            min_abs_error = torch.min(abs_error)
            median_abs_error = torch.median(abs_error)
            max_abs_error = torch.max(abs_error)

            denominator = torch.abs(A) + torch.abs(B)
            rel_error = piecewise_safe_division(abs_error, denominator)
            min_rel_error = torch.min(rel_error)
            median_rel_error = torch.median(rel_error)
            max_rel_error = torch.max(rel_error)
            
            print("Tensor Comparison Results:\n")
            print(f"Absolute Error (min/median/max):  {min_abs_error:.4g}/{median_abs_error:.4g}/{max_abs_error:.4g}")
            print(f"Relative Error (min/median/max):  {min_rel_error:.4g}/{median_rel_error:.4g}/{max_rel_error:.4g}")
    elif isinstance(A, tuple) and isinstance(B, tuple):
        if len(A) != len(B):
            raise ValueError(f"Tuple length mismatch: {len(A)} vs {len(B)}")

        for i, (a, b) in enumerate(zip(A, B)):
            print(f"Element {i}:")
            print()

            summary_stats(a, b)
            print()
    else:
        raise ValueError(f"Unknown and/or mismatching types: {type(A)} and {type(B)}")
        

In [3]:
import json
import math

import torch
from torch import nn
from torch.nn import functional as F
from torch._dynamo._trace_wrapped_higher_order_op import TransformGetItemToIndex
from typing import Optional, Tuple


MAX_POSITION_EMBEDDINGS = 131072
MAX_LENGTH = 20
TOP_K = 50
EOS_TOKEN_ID = [200002, 199999]
PAD_TOKEN_ID = 199999


class GptOssTopKRouter(nn.Module):
    def __init__(self):
        super().__init__()
        self.weight = nn.Parameter(torch.empty(32, 2880, dtype=torch.bfloat16))
        self.bias = nn.Parameter(torch.empty(32, dtype=torch.bfloat16))

    def forward(self, hidden_states):
        hidden_states = hidden_states.reshape(-1, 2880)
        router_logits = F.linear(hidden_states, self.weight, self.bias)  # (seq_len, num_experts)
        router_top_value, router_indices = torch.topk(router_logits, 4, dim=-1)  # (seq_len, top_k)
        router_top_value = torch.nn.functional.softmax(router_top_value, dim=1, dtype=router_top_value.dtype)
        router_scores = torch.zeros_like(router_logits).scatter_(1, router_indices, router_top_value)
        return router_scores, router_indices

In [4]:
inputs_and_outputs = torch.load('model.layers.7.mlp.router.pt')
inputs_and_outputs.keys()

odict_keys(['hidden_states', 'return'])

In [5]:
state_dict = torch.load('model.layers.7.mlp.router.ckpt')
state_dict.keys()

dict_keys(['bias', 'weight'])

In [6]:
module = build_module(
    GptOssTopKRouter,
    [(inputs_and_outputs['hidden_states'],)],
    checkpoint_path='model.layers.7.mlp.router.ckpt'
)
module

Neuron: Saving to compiler workdir: /tmp/nxdi_test_f35b7a63-2afb-4c81-b35c-b768e754ca5a/compiler_workdir
Neuron: Using checkpoint path: model.layers.7.mlp.router.ckpt
Neuron: Generating HLOs for the following models: ['GptOssTopKRouter']


[2025-12-06 19:42:50.314: I neuronx_distributed/parallel_layers/parallel_state.py:630] > initializing tensor model parallel with size 1
[2025-12-06 19:42:50.315: I neuronx_distributed/parallel_layers/parallel_state.py:631] > initializing pipeline model parallel with size 1
[2025-12-06 19:42:50.315: I neuronx_distributed/parallel_layers/parallel_state.py:632] > initializing context model parallel with size 1
[2025-12-06 19:42:50.316: I neuronx_distributed/parallel_layers/parallel_state.py:633] > initializing data parallel with size 1
[2025-12-06 19:42:50.316: I neuronx_distributed/parallel_layers/parallel_state.py:634] > initializing world size to 1
[2025-12-06 19:42:50.317: I neuronx_distributed/parallel_layers/parallel_state.py:379] [rank_0_pp-1_tp-1_dp-1_cp-1] Chosen Logic for replica groups ret_logic=<PG_Group_Logic.LOGIC1: (<function ascending_ring_PG_group at 0x7ff4e5f33910>, 'Ascending Ring PG Group')>
[2025-12-06 19:42:50.317: I neuronx_distributed/parallel_layers/parallel_state

Neuron: Generating 1 hlos for key: GptOssTopKRouter
Neuron: Started loading module GptOssTopKRouter
Neuron: Finished loading module GptOssTopKRouter in 0.00015592575073242188 seconds
Neuron: generating HLO: GptOssTopKRouter, input example shape = torch.Size([1, 7, 2880])
Neuron: Finished generating HLO for GptOssTopKRouter in 0.008608341217041016 seconds, input example shape = torch.Size([1, 7, 2880])
Neuron: Generated all HLOs in 0.02144002914428711 seconds
Neuron: Starting compilation for the priority HLO
Neuron: 'GptOssTopKRouter' is the priority model with bucket rank 0


2025-12-06 19:42:50.000345:  2079  INFO ||NEURON_CC_WRAPPER||: Call compiler with cmd: neuronx-cc compile --framework=XLA /tmp/nxdi_test_f35b7a63-2afb-4c81-b35c-b768e754ca5a/compiler_workdir/GptOssTopKRouter/_tp0_bk0/model.MODULE_1a742a9f70c4e493a69a+529b46ce.hlo_module.pb --output /tmp/nxdi_test_f35b7a63-2afb-4c81-b35c-b768e754ca5a/compiler_workdir/GptOssTopKRouter/_tp0_bk0/model.MODULE_1a742a9f70c4e493a69a+529b46ce.neff --target=trn1 --enable-saturate-infinity --auto-cast=none --model-type=transformer -O1 --logfile=/tmp/nxdi_test_f35b7a63-2afb-4c81-b35c-b768e754ca5a/compiler_workdir/GptOssTopKRouter/_tp0_bk0/log-neuron-cc.txt --enable-internal-neff-wrapper --verbose=35


/opt/aws_neuronx_venv_pytorch_2_8_nxd_inference/lib/python3.10/site-packages/libneuronxla/neuron_cc_wrapper.py:283: SyntaxWarning: str format compiler_flags is discouraged as its handling involves repeated joining and splitting, which can easily make mistakes if something is quoted or escaped. Use list[str] instead. Refer to documentation of the Python subprocess module for details.
  warnings.warn(SyntaxWarning(


...

Completed run_backend_driver.

Compiler status PASS
.

Neuron: Done preparing weight layout transformation
Neuron: Sharding Weights for ranks: 0...0


Completed run_backend_driver.

Compiler status PASS
[2025-12-06 19:42:54.665: I neuronx_distributed/parallel_layers/parallel_state.py:630] > initializing tensor model parallel with size 1
[2025-12-06 19:42:54.665: I neuronx_distributed/parallel_layers/parallel_state.py:631] > initializing pipeline model parallel with size 1
[2025-12-06 19:42:54.666: I neuronx_distributed/parallel_layers/parallel_state.py:632] > initializing context model parallel with size 1
[2025-12-06 19:42:54.666: I neuronx_distributed/parallel_layers/parallel_state.py:633] > initializing data parallel with size 1
[2025-12-06 19:42:54.667: I neuronx_distributed/parallel_layers/parallel_state.py:634] > initializing world size to 1
[2025-12-06 19:42:54.667: I neuronx_distributed/parallel_layers/parallel_state.py:379] [rank_0_pp-1_tp-1_dp-1_cp-1] Chosen Logic for replica groups ret_logic=<PG_Group_Logic.LOGIC1: (<function ascending_ring_PG_group at 0x7ff4e5f33910>, 'Ascending Ring PG Group')>
[2025-12-06 19:42:54.668: 

/opt/aws_neuronx_venv_pytorch_2_8_nxd_inference/lib/python3.10/site-packages/neuronx_distributed/trace/trace.py:642: UserWarning: Removing redundant keys from checkpoint: []
  warnings.warn(f"Removing redundant keys from checkpoint: {keys_to_delete}")
Neuron: Done Sharding weights in 0.013567253998189699
Neuron: NxD Model Initialized
Neuron: Finished building model in 9.324246168136597 seconds


NxDModelExecutor(
  original_name=NxDModelExecutor
  (nxd_model): RecursiveScriptModule(
    original_name=NxDModel
    (models): RecursiveScriptModule(
      original_name=ModuleDict
      (GptOssTopKRouter): RecursiveScriptModule(original_name=SPMDBucketModelScript)
    )
    (flattener_map): RecursiveScriptModule(
      original_name=ModuleDict
      (GptOssTopKRouter): JITWrapper(original_name=JITWrapper)
    )
    (packer): JITWrapper(original_name=JITWrapper)
  )
)

In [7]:
output = module(inputs_and_outputs['hidden_states'])

In [8]:
summary_stats(output, inputs_and_outputs['return'])

Element 0:

Tensor Comparison Results:

Absolute Error (min/median/max):  0/0/0.003906
Relative Error (min/median/max):  0/0/0.007721

Element 1:

Tensor Comparison Results:

Absolute Error (min/median/max):  0/0/0
Relative Error (min/median/max):  0/0/0

